# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 32 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
img_row,img_col = 120,120
img_idx = [n for n in range(0, 30, 2)] #create a list of image numbers you want to use for a particular video
x = len(img_idx) 
y, z = img_row, img_col

In [5]:
print(img_idx)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]


In [6]:
def generator(source_path, folder_list, batch_size):
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size #number of batches
        for batch in range(num_batches): # iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
    
                    #image = image.resize(250,250) ##resize
                    # crop the images and resize them. Note that the images are of 2 different shape 
                    # and the conv3D will throw error if the inputs in a batch have different shapes
                    no_rows,no_cols,no_chs=image.shape
                    if (no_rows == 120 and no_cols == 160) or (no_rows == 360 and no_cols == 360):
                        if no_rows == 120 and no_cols == 160:
                            image = image[0:120,20:140]
                        elif no_rows == 360 and no_cols == 360:
                            image = imresize(image,(120,120))
#                     print("image:",len(image))
#                     batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                    batch_data[folder,idx,:,:,0] = image[:, : , 0]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:, : , 1]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:, : , 2]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if len(folder_list)%batch_size != 0:
            batch_size = len(folder_list) - num_batches*batch_size
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    no_rows,no_cols,no_chs=image.shape
                    if (no_rows == 120 and no_cols == 160) or (no_rows == 360 and no_cols == 360):
                        if no_rows == 120 and no_cols == 160:
                            image = image[0:120,20:140]
                        elif no_rows == 360 and no_cols == 360:
                            image = imresize(image,(120,120))
#                     print("image:",len(image))
#                     batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                    batch_data[folder,idx,:,:,0] = image[:, : , 0]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:, : , 1]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:, : , 2]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
# using resnet 18
# !pip install resnet
# import resnet
# from keras.applications import ResNet50
# resnet = ResNet50(weights='imagenet', include_top=False)
# model = resnet.ResnetBuilder.build_resnet_18((img_channels, img_rows, img_cols), nb_classes)
# model.compile(loss='categorical_crossentropy', optimizer='SGD',
#               metrics=['accuracy'])

nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
# x,y,z = 30,250,250
# x,y,z = 10,100,100
input_shape=(x,y,z,3)

# Define model
model = Sequential()

# model.add(Conv3D(32,strides=(1, 1, 1),padding='same', kernel_size=(3,3,1),activation='relu',input_shape=(len(img_idx), img_row, img_col, 3)))
# model.add(MaxPooling3D(pool_size=(2, 2, 1)))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dense(5, activation='softmax'))
model.add(Conv3D(32,strides=(1, 1, 1),padding='same', kernel_size=(3,3,3),activation='relu',input_shape=(len(img_idx), img_row, img_col, 3)))
model.add(Conv3D(32,strides=(1, 1, 1),padding='same', kernel_size=(3,3,3),activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Conv3D(32,strides=(1, 1, 1),padding='same', kernel_size=(3,3,3),activation='relu'))
model.add(Conv3D(32,strides=(1, 1, 1),padding='same', kernel_size=(3,3,3),activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [9]:
optimiser = optimizers.Adam()#write your optimizer
# optimiser = optimizers.SGD(lr=0.01)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 15, 120, 120, 32)  2624      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 120, 120, 32)  27680     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 5, 40, 40, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 5, 40, 40, 32)     27680     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 5, 40, 40, 32)     27680     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 1, 13, 13, 32)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [13]:
print(steps_per_epoch,validation_steps, num_epochs )

21 4 15


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [14]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/15


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # This is added back by InteractiveShellApp.init_path()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 70s 3s/step - loss: 1.5950 - categorical_accuracy: 0.1839 - val_loss: 1.4726 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2019-09-2315_58_54.242568/model-00001-1.59592-0.18401-1.47261-0.20000.h5
Epoch 2/15
21/21 [==============================] - 38s 2s/step - loss: 1.4550 - categorical_accuracy: 0.3437 - val_loss: 1.4012 - val_categorical_accuracy: 0.4375

Epoch 00002: saving model to model_init_2019-09-2315_58_54.242568/model-00002-1.45500-0.34369-1.40118-0.43750.h5
Epoch 3/15
21/21 [==============================] - 39s 2s/step - loss: 1.2432 - categorical_accuracy: 0.4748 - val_loss: 1.4663 - val_categorical_accuracy: 0.4375

Epoch 00003: saving model to model_init_2019-09-2315_58_54.242568/model-00003-1.24882-0.47541-1.46633-0.43750.h5
Epoch 4/15
21/21 [==============================] - 32s 2s/step - loss: 1.2694 - categorical_accuracy: 0.4536 - val_loss: 1.2507 - val_categorical_accuracy: 0.5000

Epoch 00004:

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 15, 120, 120, 32)  2624      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 120, 120, 32)  27680     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 5, 40, 40, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 5, 40, 40, 32)     27680     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 5, 40, 40, 32)     27680     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 1, 13, 13, 32)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
__________